In [1]:
% ls

aclImdb/                                   poincare.ipynb
AttentionAllNeed.ipynb                     PyTorch.ipynb
attention-is-all-you-need-pytorch-master/  question-words.txt
cifar/                                     question-words.txt~
CS231n/                                    random.ipynb
Data_law/                                  senteval.ipynb
DenseNet.ipynb                             sonnet_tf_dnc.ipynb
doc2vec_imdb.ipynb                         super_resolution_with_caffe2.ipynb
doc2vec.ipynb                              tempdata/
DrQA/                                      tmp/
faces/                                     train.txt
FAST.ai_TF/                                train-v1.1.json
Fconv seq2seq.ipynb                        USC/
gensim_Trained models/                     u_s_code_scrapper.ipynb
htm_uscAll@115-95not91/                    USCpickle/
InferSent-master/                          word2vec.ipynb
infersent-pani.ipynb                       work/
new_legal_data/


In [2]:
% cd InferSent-master/encoder/

/home/piyank/Documents/Python_Scripts/InferSent-master/encoder


In [3]:
% ls

demo.ipynb          infersent.allnli.pickle  __pycache__/  samples.txt
evaluate_model.py*  models.py                README.md     xutils.py


In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint
import matplotlib

import numpy

import numpy as np
import torch

import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

GLOVE_PATH = '../dataset/GloVe/glove.840B.300d.txt'

In [5]:
model = torch.load('infersent.allnli.pickle')
model.train(False)

/home/piyank/anaconda3/lib/python3.6/site-packages/torch/serialization.py:316: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


BLSTMEncoder(
  (enc_lstm): LSTM(300, 2048, bidirectional=True)
)

In [6]:
model.set_glove_path(GLOVE_PATH)

In [24]:
sentences = []
with open('samples.txt') as f:
    for line in f:
        sentences.append(line.strip())
print(len(sentences))

9815


In [25]:
test_sentences = sentences[1000:1400]
sentences = sentences[:600]

In [27]:
answers = numpy.random.randint(0,high=2,size=(300))
test_answers = numpy.random.randint(0,high=2,size=(200))

In [28]:
model.build_vocab(sentences)

Found 2218(/2238) words with glove vectors
Vocab size : 2218


In [30]:
sent_encoded = model.encode( sentences )
test_encoded_sent = model.encode( test_sentences)

/home/piyank/Documents/Python_Scripts/InferSent-master/encoder/models.py:53: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().
  sent_output = self.enc_lstm(sent_packed)[0]  # seqlen x batch x 2*nhid


In [32]:
sent_encoded.shape

(600, 4096)

In [33]:
sent_encoded[0] , sent_encoded[1] , sent_encoded[2]

(array([ 0.10451625,  0.09512675,  0.        , ...,  0.05411991,
         0.        ,  0.03131687], dtype=float32),
 array([ 0.07801577,  0.11585563,  0.0873036 , ...,  0.00039723,
         0.        ,  0.        ], dtype=float32),
 array([ 0.0600462 ,  0.08212361,  0.03758186, ..., -0.02280346,
        -0.03814263, -0.0099379 ], dtype=float32))

In [34]:
sent_encoded = sent_encoded.reshape( sent_encoded.shape[0]//2 , -1 )
test_encoded_sent = test_encoded_sent.reshape( test_encoded_sent.shape[0]//2 , -1 )
sent_encoded.shape

(300, 8192)

In [35]:
sent_encoded[0] , sent_encoded[1]

(array([ 0.10451625,  0.09512675,  0.        , ...,  0.00039723,
         0.        ,  0.        ], dtype=float32),
 array([ 0.0600462 ,  0.08212361,  0.03758186, ...,  0.17117991,
         0.        ,  0.01631646], dtype=float32))

In [38]:
x = Variable( torch.Tensor(sent_encoded) )
x_test = Variable( torch.Tensor(test_encoded_sent) )

In [39]:
y = Variable( torch.LongTensor(answers) )
y_test = Variable( torch.LongTensor(test_answers) )

In [40]:
n1 = nn.Linear(8192,1024)
d1 = nn.Dropout(p=.5)
n2 = nn.Linear(1024,64)
d2 = nn.Dropout(p=.9)
n3 = nn.Linear(64,2)

final_layer = nn.Sequential( n1 , d1 , n2 , d2 , n3)

In [41]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( final_layer.parameters() )

In [42]:
for t in range(10):
    
    y_pred = final_layer(x)

    iter_loss = loss(y_pred, y)
    print(t, iter_loss.data[0])
    
    optimizer.zero_grad()
    iter_loss.backward()

    optimizer.step()

0 0.691713273525238
1 0.82391756772995
2 0.8211984038352966
3 0.7211378812789917
4 0.8132596015930176
5 0.8302390575408936
6 0.7096949219703674
7 0.7008435726165771
8 0.7642484307289124
9 0.7264686822891235


# Test

In [ ]:
final_layer.eval()

In [43]:
y_test_pred = final_layer( x_test )

In [46]:
y_test_pred

Variable containing:
 0.2685 -1.8548
 0.6488  0.0236
-0.7314 -0.4306
-0.5573  0.6731
-0.2402  0.3275
-0.2192 -0.5896
 0.1613 -0.5472
 0.5360 -0.3332
 0.2002 -0.2063
 0.3114  0.3549
 0.8878 -0.8207
 0.2412  0.0715
-0.5392 -0.1468
-0.8140  0.5147
 0.2565 -0.1589
 0.8424 -1.0237
-1.2985 -0.4017
 1.1588 -2.3381
 0.0228  0.0656
 0.1925  0.2493
 0.6097  0.5486
-0.2717 -0.3972
 0.0107 -0.1517
 0.9946 -0.2448
-0.2546 -0.0108
 0.3629 -0.0986
-0.4098 -1.4859
-0.4816 -0.3192
 1.2830 -0.6739
 0.3408  0.1184
 0.7535 -0.4104
 0.5120 -0.0410
 0.0818  0.0890
-0.1886  0.1968
 0.2168 -0.0698
 0.3169 -0.2603
-0.3766  0.5728
-0.9126 -0.8264
-0.1063 -0.7426
 0.1484  0.2046
 0.0463  0.1727
-0.4809 -0.1730
 0.7052  0.3047
 0.8051  0.2020
-0.4499  0.9349
 1.5366  0.7270
-0.6114 -0.8298
-0.4777 -0.5626
 0.1800 -0.4327
 0.2315 -0.3974
 0.1138  0.7910
-0.4632 -0.5557
 0.4749 -0.2313
-0.7741  0.2969
 1.2859  0.9043
-0.3306 -0.7061
-0.4783  0.3862
-0.1450 -0.0526
 1.0087  1.2582
-2.1939 -1.0056
 0.7716  0.9233
-0.

In [54]:
prob , index = F.softmax(y_test_pred).max(1)

/home/piyank/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [62]:
prob_test = prob.data.numpy()

In [63]:
index_test = index.data.numpy()

In [64]:
prob_test,index_test

(array([ 0.89314246,  0.6513961 ,  0.57463825,  0.77389717,  0.6382336 ,
         0.59155178,  0.67007071,  0.70458615,  0.60024893,  0.51088142,
         0.8466388 ,  0.54230809,  0.59687167,  0.79061085,  0.60238099,
         0.86600816,  0.71029383,  0.97060132,  0.51069236,  0.51420814,
         0.51528132,  0.53132898,  0.54051113,  0.77547348,  0.56065977,
         0.61336708,  0.74574679,  0.54052144,  0.87620002,  0.55538356,
         0.7620346 ,  0.63483042,  0.50179374,  0.59516323,  0.57115638,
         0.64041734,  0.72099698,  0.52152824,  0.65391505,  0.51404333,
         0.53155708,  0.57636327,  0.59881335,  0.64635628,  0.79975593,
         0.69202054,  0.55439317,  0.5212031 ,  0.64855433,  0.65223676,
         0.6631068 ,  0.52310801,  0.66955554,  0.74479157,  0.59425175,
         0.59278649,  0.70361406,  0.52310169,  0.56206113,  0.76644379,
         0.53785598,  0.80402774,  0.53470904,  0.62716037,  0.69745559,
         0.82974154,  0.64740229,  0.6274485 ,  0.5